<a href="https://colab.research.google.com/github/DanielleRaine/Bird-Species-Distribution-Modeling-with-Location-Information/blob/main/CLEAN_UP__ACTUAL_MLP_Loss%2BTrainer_LOCATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nbimporter
!pip install import-ipynb
!pip install torch
!pip install torcheval
!pip install torchmetrics
!pip install comet-ml

In [ ]:
import sys
import os
import json
import pickle
from typing import Any
from ctypes import sizeof

from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

import torch
import torch.utils.data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from torcheval.metrics import MeanSquaredError
from torchmetrics.regression import MeanAbsoluteError
from torchmetrics.classification import Accuracy

from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

Mounted at /content/drive


In [ ]:
class BirdHotspotsDataset(Dataset):
  """
  Dataset for Bird Species Distribution Modeling with Location Information (Bird Hotspots).
  """

  def __init__(self, features_df, targets_df):
    """
    Args:
      features_df (pd.DataFrame): DataFrame containing the features.
      targets_df (pd.DataFrame): DataFrame containing the targets.
    """

    self.data = [
        [torch.tensor(pd.to_numeric(row.drop(labels = ["hotspot_id"]).values, errors = "coerce")).squeeze(),
         torch.tensor(targets_df.loc[targets_df["hotspot_id"] == row["hotspot_id"]]
                                    .drop(columns = ["hotspot_id", "num_complete_checklists"]).values).squeeze()]
                 for i, row in features_df.iterrows()]

  def __len__(self):
    """
    Returns the length of the dataset.
    Returns:
      length (int): Length of the dataset.
    """

    return len(self.data)

  def __getitem__(self, idx):
    """
    Returns the record and its target.
    Args:
      idx (int): Index of the record.
    Returns:
      record (torch.Tensor): Record.
      target (torch.Tensor): Target.
    """

    return self.data[idx][0], self.data[idx][1]

In [ ]:
# Data that includes location data
df_train = pd.read_csv('/content/drive/MyDrive/TeamMila/ProjectDataset/normalized_train_loc.csv')
df_val = pd.read_csv('/content/drive/MyDrive/TeamMila/ProjectDataset/normalized_val_loc.csv')
df_test = pd.read_csv('/content/drive/MyDrive/TeamMila/ProjectDataset/normalized_test_loc.csv')
df_targets = pd.read_csv('/content/drive/MyDrive/TeamMila/ProjectDataset/targets.csv')

In [ ]:
# Columns that will be used for the data
df_columns = ["hotspot_id", "lon", "lat"] + [f"bio_{i}" for i in range(1, 20)] + ["orcdrc", "phihox", "cecsol", "bdticm", "clyppt", "sltppt", "sndppt", "bldfie"]

In [ ]:
df_train = df_train[df_columns]
df_val = df_val[df_columns]
df_test = df_test[df_columns]

In [ ]:
train_dataset = BirdHotspotsDataset(df_train, df_targets)
valid_dataset = BirdHotspotsDataset(df_val, df_targets)
test_dataset = BirdHotspotsDataset(df_test, df_targets)

In [ ]:
# Pickle each dataset
with open('/content/drive/MyDrive/TeamMila/ProjectDataset/train_dataset_loc.p', 'wb') as f:
    pickle.dump(train_dataset, f)

with open('/content/drive/MyDrive/TeamMila/ProjectDataset/valid_dataset_loc.p', 'wb') as f:
    pickle.dump(valid_dataset, f)

with open('/content/drive/MyDrive/TeamMila/ProjectDataset/test_dataset_loc.p', 'wb') as f:
    pickle.dump(test_dataset, f)

In [ ]:
with open('/content/drive/MyDrive/TeamMila/ProjectDataset/train_dataset_loc.p', "rb") as f:
  train_dataset = pickle.load(f)

with open('/content/drive/MyDrive/TeamMila/ProjectDataset/valid_dataset_loc.p', "rb") as f:
  valid_dataset = pickle.load(f)

with open('/content/drive/MyDrive/TeamMila/ProjectDataset/test_dataset_loc.p', "rb") as f:
   test_dataset = pickle.load(f)

In [ ]:
# different datloaders for different splits of the data
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
class EncounterRateMLP(nn.Module):
    def __init__(self, env_input_size=27, loc_input_size=2, num_classes=1, hidden_dimensions=128):
        super(EncounterRateMLP, self).__init__()
        self.inc_bias = False
        # Encoder for environmental features
        self.env_encoder = nn.Sequential(
            nn.Linear(env_input_size, hidden_dimensions),
            nn.LeakyReLU(inplace=True),
            nn.Linear(hidden_dimensions, hidden_dimensions),
            nn.LeakyReLU(inplace=True)
        )
        # Encoder for location features
        self.loc_encoder = nn.Sequential(
            nn.Linear(loc_input_size, hidden_dimensions),
            nn.LeakyReLU(inplace=True),
            nn.Linear(hidden_dimensions, hidden_dimensions),
            nn.LeakyReLU(inplace=True)
        )
        # Final output layer after concatenating encoded features
        self.output_layer = nn.Linear(hidden_dimensions * 2, num_classes)
    def forward(self, env_features, loc_features):
        # Pass through environmental encoder
        env_encoded = self.env_encoder(env_features)
        # Pass through location encoder
        loc_encoded = self.loc_encoder(loc_features)
        # Concatenate encoded features
        combined_features = torch.cat((env_encoded, loc_encoded), dim=1)
        # Pass through final output layer and apply sigmoid
        output = torch.sigmoid(self.output_layer(combined_features))
        return output




In [ ]:
def TopKAccuracy(outputs, targets, k = None):
    if k is None:
        # Let K be the number of nonzero values for a set of predictions
        sum_correct = 0
        batch_size = outputs.shape[0]
        for output, target in zip(outputs, targets):
            k = torch.count_nonzero(target).item()
            top_k_preds = torch.topk(output, k, dim=0).indices
            true_labels = torch.topk(target, k, dim=0).indices
            sum_correct += torch.any(top_k_preds == true_labels)
        correct = sum_correct / batch_size
        return correct

    # Get the top K predictions' indices
    top_k_preds = torch.topk(outputs, k, dim=1).indices
    true_labels = torch.topk(targets, k, dim=1).indices

    # Check if the true label is in the top K predictions
    correct = torch.any(top_k_preds == true_labels, dim=1)

    # Return the average success of the batch of predictions
    return correct.float().mean().item()

In [ ]:
def training_step(model, dataloader, eval_dataloader, criterion, optimizer, device, checkpoint_dir, num_epochs, experiment = None):
    model.to(device)
    best_loss = float('inf')
    last_checkpoint_path = os.path.join(checkpoint_dir, 'last_checkpoint.pth')
    best_checkpoint_path = os.path.join(checkpoint_dir, 'best_checkpoint.pth')
    mse_metric = MeanSquaredError().to(device)
    mae_metric = MeanAbsoluteError().to(device)
    mse_metric_eval = MeanSquaredError().to(device)
    mae_metric_eval = MeanAbsoluteError().to(device)

    for epoch in range(num_epochs):
        model.train()
        mse_metric.reset()
        mae_metric.reset()
        mse_metric_eval.reset()
        mae_metric_eval.reset()

        print("A NEW EPOCH HAS STARTED")

        running_loss = 0.0

        top_10_correct = 0
        top_30_correct = 0
        top_k_correct = 0
        num_batches = 0

        top_10_correct_eval = 0
        top_30_correct_eval = 0
        top_k_correct_eval = 0
        num_batches_eval = 0

        for inputs, targets in dataloader:
          ### runtime error of having inputs and targets as non-floats
          inputs, targets = inputs.float().to(device), targets.float().to(device)

          optimizer.zero_grad()

          outputs = model(inputs[:, 2:], inputs[:, :2])

          #runtime error address: targets not in between 0 to 1
          targets = torch.clamp(targets,0,1)
          loss = criterion(outputs, targets)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()

          mse_metric.update(outputs, targets)
          mae_metric.update(outputs, targets)

          top_10_correct += TopKAccuracy(outputs, targets, k = 10)
          top_30_correct += TopKAccuracy(outputs, targets, k = 30)
          top_k_correct += TopKAccuracy(outputs, targets)
          num_batches += 1

        model.eval()

        print("EVAL")

        for inputs, targets in eval_dataloader:
          with torch.no_grad():
            inputs, targets = inputs.float().to(device), targets.float().to(device)

            outputs = model(inputs[:, 2:], inputs[:, :2])
            targets = torch.clamp(targets,0,1)

            mse_metric_eval.update(outputs, targets)
            mae_metric_eval.update(outputs, targets)

            top_10_correct_eval += TopKAccuracy(outputs, targets, k = 10)
            top_30_correct_eval += TopKAccuracy(outputs, targets, k = 30)
            top_k_correct_eval += TopKAccuracy(outputs, targets)
            num_batches_eval += 1

        mse = mse_metric.compute()
        mae = mae_metric.compute()

        top_10 = top_10_correct / num_batches
        top_30 = top_30_correct / num_batches
        top_k = top_k_correct / num_batches

        mse_eval = mse_metric_eval.compute()
        mae_eval = mae_metric_eval.compute()

        top_10_correct_eval = top_10_correct_eval / num_batches_eval
        top_30_correct_eval = top_30_correct_eval / num_batches_eval
        top_k_correct_eval = top_k_correct_eval / num_batches_eval

        if experiment is not None:
          experiment.log_metrics({
              "mse": mse,
              "mae": mae,
              "top_10_accuracy": top_10,
              "top_30_accuracy": top_30,
              "top_k_accuracy": top_k,
              "mse_eval": mse_eval,
              "mae_eval": mae_eval,
              "top_10_accuracy_eval": top_10_correct_eval,
              "top_30_accuracy_eval": top_30_correct_eval,
              "top_k_accuracy_eval": top_k_correct_eval
              }, step=epoch
          )

        print(f"Epoch [{epoch + 1}/{num_epochs}], " f"Mean Squared Error: {mse.item():.5f}, " f"Mean Absolute Error: {mae.item():.5f}, " f"Top 10 Accuracy: {top_10:.5f}, " f"Top 30 Accuracy: {top_30:.5f}," f" Top K Accuracy: {top_k:.5f}")

        #-----
        #below statement has been tested and does work
        # torch.save(model.state_dict(), last_checkpoint_path)
        #-----

        checkpoint = {
        'epoch': epoch + 1,  # Save the epoch number (1-based index)
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss.item()
        }
        #this should save a checkpoint after every epoch [HAVE NOT TESTED]
        torch.save(checkpoint, f"checkpoints_epoch_{epoch+1}.pth")

    epoch_loss = running_loss / len(dataloader)
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), best_checkpoint_path)
        print(f'CURRENT BEST MODEL: {epoch + 1} LOSS: {best_loss:.5f}')


    print(f'CURRENT EPOCH: [{epoch + 1}/{num_epochs}], LOSS: {epoch_loss:.5f}')


In [ ]:
def testing_step(model, dataloader, device, experiment = None):
    model.to(device)

    mse_metric = MeanSquaredError().to(device)
    mae_metric = MeanAbsoluteError().to(device)

    mse_metric.reset()
    mae_metric.reset()

    top_10_correct = 0
    top_30_correct = 0
    top_k_correct = 0
    num_batches = 0

    model.eval()

    for inputs, targets in dataloader:
      inputs, targets = inputs.float().to(device), targets.float().to(device)
      outputs = model(inputs[:, 2:], inputs[:, :2])
      targets = torch.clamp(targets,0,1)

      mse_metric.update(outputs, targets)
      mae_metric.update(outputs, targets)

      top_10_correct += TopKAccuracy(outputs, targets, k = 10)
      top_30_correct += TopKAccuracy(outputs, targets, k = 30)
      top_k_correct += TopKAccuracy(outputs, targets)
      num_batches += 1

    mse = mse_metric.compute()
    mae = mae_metric.compute()

    top_10 = top_10_correct / num_batches
    top_30 = top_30_correct / num_batches
    top_k = top_k_correct / num_batches

    print(f"Mean Squared Error: {mse.item():.5f}, Mean Absolute Error: {mae.item():.5f}")

    print(f"Top 10 Accuracy: {top_10:.5f}, Top 30 Accuracy: {top_30:.5f}, Top K Accuracy: {top_k:.5f}")

    if experiment is not None:
      experiment.log_metrics({
          "mse_test": mse,
          "mae_test": mae,
          "top_10_accuracy_test": top_10,
          "top_30_accuracy_test": top_30,
          "top_k_accuracy_test": top_k
      })

In [ ]:
def CrossEntropyLoss(model,predictions,targets):
  criterion = nn.BCELoss()
  loss = criterion(predictions, targets)
  return loss


In [ ]:
# Use 'Secrets' tab to store real api_key value

experiment = Experiment(
    api_key=userdata.get('comet_api_key'),
    project_name="bird-species-distribution-modeling-with-location-information",
    workspace="danielleraine"
)

experiment.set_name("baseline-loc-augnorm")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/danielleraine/bird-species-distribution-modeling-with-location-information/82bddc364f274f99a0f156c78581e0a7

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [ ]:
hyper_params = {
    'batch_size': 64,
    'learning_rate': 0.001,
    'num_epochs': 10,
}

experiment.log_parameters(hyper_params)

In [ ]:
model = EncounterRateMLP(27, 2, 671, hidden_dimensions=128).float()
criterion = nn.BCELoss()

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

last_checkpoint_path = os.path.join(checkpoint_dir, 'last_checkpoint.pth')
best_checkpoint_path = os.path.join(checkpoint_dir, 'best_checkpoint.pth')

# Save model state after each epoch
# torch.save(model.state_dict(),
#            last_checkpoint_path)
loss = 0.0
epoch = 0
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, "best_checkpoint.pth")

training_step(
    model=model,
    dataloader=train_dataloader,
    eval_dataloader=valid_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    checkpoint_dir=checkpoint_dir,
    num_epochs=10,  # Number of epochs for training
    experiment=experiment
)

A NEW EPOCH HAS STARTED
EVAL
Epoch [1/10], Mean Squared Error: 0.00994, Mean Absolute Error: 0.02764, Top 10 Accuracy: 0.73379, Top 30 Accuracy: 0.81033, Top K Accuracy: 0.82360
A NEW EPOCH HAS STARTED
EVAL
Epoch [2/10], Mean Squared Error: 0.00747, Mean Absolute Error: 0.02408, Top 10 Accuracy: 0.75475, Top 30 Accuracy: 0.83624, Top K Accuracy: 0.84617
A NEW EPOCH HAS STARTED
EVAL
Epoch [3/10], Mean Squared Error: 0.00728, Mean Absolute Error: 0.02356, Top 10 Accuracy: 0.75935, Top 30 Accuracy: 0.83863, Top K Accuracy: 0.85077
A NEW EPOCH HAS STARTED
EVAL
Epoch [4/10], Mean Squared Error: 0.00717, Mean Absolute Error: 0.02327, Top 10 Accuracy: 0.75937, Top 30 Accuracy: 0.83860, Top K Accuracy: 0.85208
A NEW EPOCH HAS STARTED
EVAL
Epoch [5/10], Mean Squared Error: 0.00708, Mean Absolute Error: 0.02305, Top 10 Accuracy: 0.76226, Top 30 Accuracy: 0.84157, Top K Accuracy: 0.85230
A NEW EPOCH HAS STARTED
EVAL
Epoch [6/10], Mean Squared Error: 0.00703, Mean Absolute Error: 0.02291, Top 10 A

In [ ]:
# checkpoint = torch.load("checkpoints/best_checkpoint.pth")
# print(checkpoint.keys())
# print(checkpoint['feats.0.weight'])

NameError: name 'torch' is not defined

In [ ]:
# torch.save(model, "/content/drive/MyDrive/TeamMila/Models/baseline-loc-augnorm.pth")

In [ ]:
# model = torch.load("/content/drive/MyDrive/TeamMila/Models/baseline_model.pth", weights_only=False)

In [ ]:
testing_step(model, test_dataloader, device, experiment)

Mean Squared Error: 0.00977, Mean Absolute Error: 0.02830
Top 10 Accuracy: 0.70994, Top 30 Accuracy: 0.79737, Top K Accuracy: 0.80195


In [ ]:
log_model(experiment, model, "baseline-loc-augnorm")

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline-loc-augnorm
COMET INFO:     url                   : https://www.comet.com/danielleraine/bird-species-distribution-modeling-with-location-information/82bddc364f274f99a0f156c78581e0a7
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [1337]               : (0.04721482843160629, 28.364704132080078)
COMET INFO:     mae [10]                  : (0.022451551631093025, 0.02771819196641445)
COMET INFO:     mae_eval [10]             : (0.024273058399558067, 0.026930954307317734)
COMET INFO:     mae_test                  : 0.02686145529150963
COMET INFO:     mse [10]                  : (0.006865320727229118, 0.010266482830047607)
COMET INF